<a href="https://colab.research.google.com/github/jewoolee0502/ComputerVision/blob/main/ECSE415_A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECSE 415: Introduction to Computer Vision
###### Jewoo Lee - 260910789
###### Anthony Bonta - 261053688

## Assignment 5: Video Analysis

### Library Requirements

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Installs
!pip install -q ultralytics
!pip install -q kaggle

# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
import math
import torch

from ultralytics import YOLO

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


#### Path

In [2]:
path = '/content/drive/MyDrive/McGill/2025/Fall 2025/ECSE 415/A5/' # jay's path
# path = '' # anthony's path

object_tracking_root = os.path.join(path, "Object_Tracking")

### 1. Data Preparation

In [3]:
task1_images_dir = os.path.join(object_tracking_root, "Task1", "images")
task1_video_path = os.path.join(object_tracking_root, "task1_input.mp4")

print("Task1 images dir:", task1_images_dir)
print("Output video path:", task1_video_path)

Task1 images dir: /content/drive/MyDrive/McGill/2025/Fall 2025/ECSE 415/A5/Object_Tracking/Task1/images
Output video path: /content/drive/MyDrive/McGill/2025/Fall 2025/ECSE 415/A5/Object_Tracking/task1_input.mp4


In [7]:
FPS = 14 # given fps value

def images_to_video(images_dir, output_path, fps):
  # all images are .jpg
  image_files = sorted(glob.glob(os.path.join(images_dir, "*.jpg")))
  print(f"Found {len(image_files)} images in {images_dir}")

  if len(image_files) == 0:
    raise RuntimeError(f"No .jpg image files found in {images_dir}")

  # read the first image and get its dimensions
  first_img = cv2.imread(image_files[0])
  height, width = first_img.shape[:2]
  frame_size = (width, height)
  print(f"Target frame size: {frame_size}")

  # set up the video writer
  fourcc = cv2.VideoWriter_fourcc(*"mp4v")
  writer = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

  if not writer.isOpened():
    raise RuntimeError(f"VideoWriter could not be opened for {output_path}")

  # write all images as frames
  for idx, img_path in enumerate(image_files):
    frame = cv2.imread(img_path)

    if frame is None:
      print(f"Skipping unreadable image: {img_path}")
      continue

    # resize every frame matches the first image's size
    frame = cv2.resize(frame, frame_size)
    writer.write(frame)

  writer.release()
  print(f"Video saved to: {output_path}")

images_to_video(task1_images_dir, task1_video_path, FPS)

Found 429 images in /content/drive/MyDrive/McGill/2025/Fall 2025/ECSE 415/A5/Object_Tracking/Task1/images
Target frame size: (1920, 1080)
Video saved to: /content/drive/MyDrive/McGill/2025/Fall 2025/ECSE 415/A5/Object_Tracking/task1_input.mp4


### 2. Model Implementation

### 3. Model Evaluation

### 4. Prediction & Kaggle Competition